# Analyzing Motorcycle Part Sales with SQL

You're working for a company that sells motorcycle parts, and they've asked for some help in analyzing their sales data!

They operate three warehouses in the area, selling both retail and wholesale. They offer a variety of parts and accept credit cards, cash, and bank transfer as payment methods. However, each payment type incurs a different fee.

The board of directors wants to gain a better understanding of wholesale revenue by product line, and how this varies month-to-month and across warehouses. You have been tasked with calculating net revenue for each product line and grouping results by month and warehouse. The results should be filtered so that only `"Wholesale"` orders are included.

They have provided you with access to their database, which contains the following table called `sales`:

## Sales
| Column | Data type | Description |
|--------|-----------|-------------|
| `order_number` | `VARCHAR` | Unique order number. |
| `date` | `DATE` | Date of the order, from June to August 2021. |
| `warehouse` | `VARCHAR` | The warehouse that the order was made from&mdash; `North`, `Central`, or `West`. |
| `client_type` | `VARCHAR` | Whether the order was `Retail` or `Wholesale`. |
| `product_line` | `VARCHAR` | Type of product ordered. |
| `quantity` | `INT` | Number of products ordered. | 
| `unit_price` | `FLOAT` | Price per product (dollars). |
| `total` | `FLOAT` | Total price of the order (dollars). |
| `payment` | `VARCHAR` | Payment method&mdash;`Credit card`, `Transfer`, or `Cash`. |

In [1]:
pip install duckdb

  Obtaining dependency information for duckdb from https://files.pythonhosted.org/packages/7c/d5/7688cdc1b57c9ada17b6f6143a923606294b462e4b15e3ac249c04649bde/duckdb-0.9.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 53.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import duckdb
import pandas as pd

In [3]:
sales = pd.read_csv("/kaggle/input/portfolio-projects-data/sales_data.csv")

In [4]:
query1 = """
		SELECT
			product_line,
			FORMAT(date, 'MMMM') AS Month,
			warehouse,
			ROUND(SUM(total)::numeric,2) AS net_revenue
		FROM sales
		WHERE client_type = 'Wholesale'
		GROUP BY product_line,  warehouse, Month
		ORDER BY product_line, Month, net_revenue DESC;
  """
  
result1 = duckdb.query(query1).to_df()
result1

,product_line,Month,warehouse,net_revenue
0,Breaking system,2021-06-05,Central,54.60
1,Breaking system,2021-06-08,West,1212.77
2,Breaking system,2021-06-08,Central,830.67
3,Breaking system,2021-06-09,Central,594.24
4,Breaking system,2021-06-11,Central,404.93
...,...,...,...,...
208,Suspension & traction,2021-08-22,Central,609.01
209,Suspension & traction,2021-08-24,North,1036.93
210,Suspension & traction,2021-08-24,Central,973.74
211,Suspension & traction,2021-08-25,North,259.12


In [5]:
# /* Find the monthly total net revenue made from each warehouse for retail and wholesale seperately */

query2 = """
		SELECT
			FORMAT(date, 'MMMM') AS Month,
			warehouse,
			client_type,
			ROUND(SUM(total)::numeric,2) AS net_revenue
		FROM sales
		GROUP BY Month, warehouse, client_type
		ORDER BY 1 DESC,2, 3;"""

result2 = duckdb.query(query2).to_df()
result2

,Month,warehouse,client_type,net_revenue
0,2021-08-28,Central,Retail,295.83
1,2021-08-28,Central,Wholesale,1151.57
2,2021-08-28,North,Retail,705.92
3,2021-08-28,North,Wholesale,548.13
4,2021-08-28,West,Retail,170.21
...,...,...,...,...
390,2021-06-01,Central,Retail,934.10
391,2021-06-01,North,Retail,709.67
392,2021-06-01,North,Wholesale,2100.24
393,2021-06-01,West,Retail,54.41


In [6]:
# /* Find the total revenue by product line highest to lowest*/

query3 = """
			SELECT
				product_line,
				ROUND(SUM(total)::numeric,2) AS net_revenue
			FROM sales
			GROUP BY product_line
			ORDER BY 2 DESC;
   """
   
result3 = duckdb.query(query3).to_df()
result3

,product_line,net_revenue
0,Suspension & traction,73014.21
1,Frame & body,69024.73
2,Electrical system,43612.71
3,Breaking system,38350.15
4,Engine,37945.38
5,Miscellaneous,27165.82


In [7]:
# /* Find the relative percentage of total revenue by warehouse */

query4 = """
		WITH cte_revenue AS(
			SELECT
				warehouse,
				ROUND(SUM(total)::numeric,2) AS net_revenue
			FROM sales
			GROUP BY warehouse
		),
		cte_totalrevenue AS(
			SELECT 
				SUM(net_revenue) AS total_revenue
			FROM cte_revenue
		)
		SELECT
			r.warehouse,
			r.net_revenue,
			ROUND((r.net_revenue / tr.total_revenue) * 100, 2) AS revenue_percentage
		FROM cte_revenue r
		CROSS JOIN cte_totalrevenue tr
		ORDER BY 3 DESC;
"""

result4 = duckdb.query(query4).to_df()
result4

,warehouse,net_revenue,revenue_percentage
0,Central,141982.88,49.11
1,North,100203.63,34.66
2,West,46926.49,16.23


In [8]:
# /* Find the distribution of payment methods according to total price (make small, medium and large purchase bins) as relative percentage */

query5 = """
			SELECT
				payment,
				CASE 
					WHEN total < 100 THEN 'small'
					WHEN total < 1000 AND total >= 100 THEN 'medium'
					WHEN total >= 1000 THEN 'large' END AS purchase_size,
				COUNT(payment) AS payment_m
			FROM sales
			GROUP BY payment, purchase_size
			ORDER BY 1,2;
"""

result5 = duckdb.query(query5).to_df()
result5

,payment,purchase_size,payment_m
0,Cash,medium,74
1,Cash,small,42
2,Credit card,medium,427
3,Credit card,small,232
4,Transfer,large,57
5,Transfer,medium,159
6,Transfer,small,9
